In [1]:
import pandas as pd

In [7]:
questions = 0
lecture = 1

In [2]:
lectures = pd.read_csv('lectures.csv')
questions = pd.read_csv('questions.csv')
train = pd.read_csv('train.csv')

In [5]:
# Изменим тип prior_questuin_had_explanation на float
train["prior_question_had_explanation"] = train.prior_question_had_explanation.astype(float)

In [8]:
# Используем dropna() для "вопросов",
# исходя из описания для prior_question_had_explanation, где указано,
# что это вопросы без обратной связи, используемые для диагностики.
train_questions = train[train.content_type_id == questions].dropna()

In [9]:
# Уберём лишние колонки из train таблицы для лекций.
not_required_columns = [
    "prior_question_had_explanation",
    "prior_question_elapsed_time", "answered_correctly",
    "user_answer", "timestamp"
]
train_lectures = train[
    train.content_type_id == lecture
].drop(
    not_required_columns, axis=1
)

In [10]:
# Проанализуем полное описание полученной таблицы
main_columns = [
        "timestamp", "answered_correctly",
        "prior_question_elapsed_time",
        "prior_question_had_explanation"
    ]
train_questions.filter(main_columns).describe()

,timestamp,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
count,9.887879e+07,9.887879e+07,9.887879e+07,9.887879e+07
mean,7.732632e+09,6.571448e-01,2.542381e+04,9.070252e-01
std,1.160779e+10,4.746636e-01,1.994815e+04,2.903971e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.302152e+08,0.000000e+00,1.600000e+04,1.000000e+00
50%,2.688358e+09,1.000000e+00,2.100000e+04,1.000000e+00
75%,9.988412e+09,1.000000e+00,2.966600e+04,1.000000e+00
max,8.742577e+10,1.000000e+00,3.000000e+05,1.000000e+00


In [11]:
# region Зависимость правильности ответов от времени, затраченного на ответ
train_questions.groupby(
            ["answered_correctly"], as_index=False
        ).aggregate(
            {'prior_question_elapsed_time': 'mean'}
        ).sort_values(
            "answered_correctly"
        )

,answered_correctly,prior_question_elapsed_time
0,0,25641.991586
1,1,25309.977185


Явной корреляции не наблюдается

In [12]:
# region Зависимость правильности ответа от чтения пояснений к вопросам
train_questions.filter(
        ["prior_question_had_explanation", "answered_correctly"]
    ).groupby(
        'prior_question_had_explanation'
    ).aggregate(
        {'answered_correctly': 'mean'}
    )

,answered_correctly
prior_question_had_explanation,
0.0,0.500629
1.0,0.673189


Вывод: Пользователь, прочитавший пояснение к предыдущему блоку вопросов, с большей вероятность верно ответит на следующий вопрос

In [13]:
# Проверим эту теорию про чтение к текущим вопросами
train_questions["current_question_explained"] = train_questions.prior_question_had_explanation.shift(1)

train_questions.filter(
    ["current_question_explained", "answered_correctly"]
).groupby(
    'current_question_explained'
).aggregate(
    {'answered_correctly': 'mean'}
)


,answered_correctly
current_question_explained,
0.0,0.503717
1.0,0.672872


Вывод: После правильного ответа пользователи чаще смотрят пояснения, чем после неправильного